In [1]:
from wilcoxon import geo
from wilcoxon import sheets
from wilcoxon import spiderman
from math import sin, cos, atan2, radians, sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import pandas as pd
import geopandas as gpd
import geoplot
from abc import ABC, abstractmethod
import pyarrow.parquet as pq
from shapely.ops import transform
from functools import partial
import shapely
import re
import fiona
import json
import requests
import operator
import os
import datetime as dt
import time
import pyproj

%matplotlib inline

In [4]:
class ElevationMap:
    
    PATH_TO_ELEVATION_MAP = "../Geospatial/GEOSPATIAL/national-map-line/national-map-line-geojson.geojson"
    
    __slots__ = ("ELEVATION_MAP", "OUTLINE")
    
    def __init__(self):
        print("Generating Elevation Map...")
        self.ELEVATION_MAP = self._generate_elevation_map()
        self.OUTLINE = self.ELEVATION_MAP.geometry[self.ELEVATION_MAP.alt == 0][0]
        print("Complete")
        
    def __str__(self):
        return "Singapore object"
        
    def _generate_elevation_map(self):
        elevation_df = gpd.read_file(self.PATH_TO_ELEVATION_MAP)
        elevation_df = elevation_df[elevation_df.Description.str.contains(r"<th>NAME</th> <td>\d+</td>")]
        elevation_df["Elevation"] = elevation_df.Description.str.extract(r"<th>NAME</th> <td>(\d+)</td>")

        list_of_polygons = []
        for geometry in elevation_df.geometry.tolist():
            try:
                list_of_polygons.append(
                    shapely.geometry.Polygon(
                        [(float(i[0]), float(i[1])) for i in re.findall(r"(\d+\.\d+)\s(\d+\.\d+)", str(geometry))]
                    )
                )
            except ValueError:
                list_of_polygons.append("")
                
        elevation_df["Polygon"] = list_of_polygons
        elevation_df = elevation_df[elevation_df.Polygon != ""]
        multipolygons = []
        
        for alt in elevation_df.Elevation.unique():
            multipolygons.append({
                "alt": int(alt),
                "geometry": shapely.geometry.MultiPolygon(
                     elevation_df[elevation_df.Elevation == alt].Polygon.tolist())
                })
            
        return gpd.GeoDataFrame(multipolygons)
    
    def _generate_train_exits(self):
        exits_df = gpd.read_file(Singapore.path_to_TrainStationExits)
        exits_df["geometry"] = exits_df.geometry.to_crs(epsg=4326)
        exits_dict = exits_df.to_dict("index")
    
class DistanceCalculator:
    
    @staticmethod
    def get_distance_between(p1, p2):
        if type(p1) != shapely.geometry.Point or type(p2) != shapely.geometry.Point:
            raise ValueError("p1 and p2 must both be of type shapely.geometry.Point")
        return DistanceCalculator.get_distance_between_xy(p1.y, p1.x, p2.y, p2.x)

    @staticmethod
    def get_distance_between_xy(lat1, lon1, lat2, lon2):
        R = 6371
        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        return round(R*2*atan2(sqrt(a), sqrt(1-a)), 3)
    

In [5]:
class Features(ABC):
    
    def __init__(self, list_of_features, type_="features"):
        self.type_ = type_
        setattr(self, self.type_, {feature.name: feature for feature in list_of_features})
          
    @abstractmethod
    def __str__(self):
        pass
    
    @abstractmethod
    def __repr__(self):
        pass
    
    def __getitem__(self, search_term):
        if search_term in self.features:
            return self.features[search_term]
        else:
            search_results = dict(filter(
                lambda feature: search_term in feature[0],
                self.features.items()))
            if len(search_results) == 1:
                return list(search_results)[0]
            else:
                return search_results
        
    def __len__(self):
        return len(self.features)
    
    @property
    def features(self):
        return getattr(self, self.type_)
    
    @property
    def keys(self):
        return list(vars(self).keys())
    
    def get_with_regex(self, search_term=""):
        search_results = dict(filter(
            lambda feature: re.match(search_term, feature[0]),
            self.features.items()))
        if len(search_results) == 1:
            return list(search_results)[0]
        else:
            return search_results
    
    def set_elevation(self):
        for name, feature in self.features.items():
            feature.set_elevation()
    
    def get_nearest_to_target(self, target):    
        n_distance = float("inf")
        n_feature = None
        for name, feature in self.features.items():
            distance = target.get_distance_to(feature)
            if distance < n_distance:
                n_distance = distance
                n_feature = feature
        target.nearest_dict[n_feature.type_] = {"feature": n_feature, "dist": n_distance}
        return n_distance, n_feature
    
    def map_nearest_to_targets(self, targets):
        for name, feature in self.features.items():
            s_nearest = "Nearest " + str(type(targets)).split(".")[1].split("'")[0][:-1]
            s_distance = "Distance to " + s_nearest
            n_distance, n_target = targets.get_nearest_to_target(feature)
            feature.nearest_dict[n_target.type_] = {"target": n_target, "dist": n_distance}
    
    def sort_by(
            self, by="name",
            desc=True, ignore_zero=True,
            ignore_na=True, return_=True,
            in_place=True):
        for name, feature in self.features.items():
            feature.sort_key = by
        sorted_dict = dict(sorted(
                self.features.items(),
                key=lambda item: item[1],
                reverse=desc))
        
        if in_place:
            setattr(self, self.type_, sorted_dict)
        
        if return_:
            sorted_list = []
            for name, feature in self.features.items():
                attr_value = getattr(feature, by)
                if (ignore_zero or attr_value!=0) and (ignore_na or attr_value is not None):
                    sorted_list.append((name, attr_value))
            return sorted_list
                
    def get_attribute(self, attr, as_="dict"):
        try:
            if as_ == "dict":
                return {name: getattr(feature, attr) for name, feature in self.features.items()}
            elif as_ == "list":
                return [getattr(feature, attr) for name, feature in self.features.items()]
            else:
                raise ValueError(f"""return_ must be either 'dict', 'feature' or 'dist'.
                                     Value given was {return_}""")
        except AttributeError:
            raise AttributeError(f"")
        
    def get_nearest(self, target_type, as_="dict", all_=True):
        if all_:
            return {name: feature.nearest_dict for name, feature in self.features.items()}
        else:
            return {name: feature.nearest_dict["target_type"] for name, feature in self.features.items()}

class Feature(ABC):
    
    __slots__ = ("name", "sort_key", "type_", "long", "lat", "geometry", "elevation", "nearest_dict")
    ELEVATION_MAP = ElevationMap().ELEVATION_MAP
    
    def __init__(self, name, long=np.nan, lat=np.nan, geometry=np.nan, type_="feature"):
        self.name = name
        self.sort_key = "name"
        self.type_ = type_
        self.nearest_dict = {}
        
        if not lat+long > 0:
            self.long, self.lat = self.set_coords(self.name)   
        else:
            self.lat = lat
            self.long = long
        
        if type(geometry) == shapely.geometry.polygon.Polygon:
            self.geometry = Shape(geometry)
        elif type(geometry) == shapely.geometry.linestring.LineString:
            self.geometry = Line(geometry)
        else:
            self.geometry = Point(self.point)
            
    @abstractmethod
    def __str__(self):
        pass
    
    @abstractmethod
    def __repr__(self):
        pass
    
    def __lt__(self, feature):
        if self.sort_key != feature.sort_key:
            raise AssertionError(
                f"""{self.name} with sort key {self.sort_key} is incomparable against 
                {feature.name} with sort key {feature.sort_key}.
                """)
        else:
            return getattr(self, self.sort_key) < getattr(feature, feature.sort_key)
    
    def __gt__(self, feature):
        if self.sort_key != feature.sort_key:
            raise AssertionError(
                f"""{self.name} with sort key {self.sort_key} is incomparable against 
                {feature.name} with sort key {feature.sort_key}.
                """)
        else:
            return getattr(self, self.sort_key) < getattr(feature, feature.sort_key)
    
    @property
    def point(self):
        return shapely.geometry.Point(self.long, self.lat)
    
    def set_coords(self, name):
        try:
            search_term = "+".join(name.split())
            coords = json.loads(requests.get(
                f"""https://developers.onemap.sg/commonapi
                    /search?searchVal={search_term}
                    &returnGeom=Y&getAddrDetails=Y""").text)["results"][0]
            long, lat = float(coords["LONGITUDE"]), float(coords["LATITUDE"])
            return (long, lat)
        except IndexError:
            print(name, "could not be found.")
            return (np.nan, np.nan)
        
    def set_elevation(self):
        n_distance_dict = {}
        max_elevation = 0
        
        for elevation, multipolygon in self.ELEVATION_MAP.values:
            n_distance_dict[int(elevation)] = min(
                [polygon.exterior.distance(self.point) for polygon in list(multipolygon.geoms)]
            )
            for polygon in list(multipolygon.geoms):
                if self.point.within(polygon):
                    max_elevation = elevation

        self.elevation = (
            (max_elevation+20)*n_distance_dict[max_elevation] 
            + (max_elevation)*n_distance_dict[max_elevation+20]
            ) / (
            n_distance_dict[max_elevation]
            + n_distance_dict[max_elevation+20]
            )
    
    def get_distance_to(self, feature):
        return min(
            self.geometry.get_distance_to_feature(feature),
            feature.geometry.get_distance_to_feature(self))
    
    def get_distance_to_nearest_target(self, targets):
        n_distance, n_feature = targets.get_nearest_to_target(self)
        self.nearest_dict[n_feature.type_] = {"feature": n_feature, "dist": n_distance}
        return n_distance
    
    def get_nearest(self, target_type_, return_="dict"):
        try:
            if return_ == "dict":
                return self.nearest_dict[target_type_]
            elif return_ == "feature" or return_ == "key":
                return self.nearest_dict[target_type_]["feature"]
            elif return_ == "dist" or return_ == "value":
                return self.nearest_dict[target_type_]["dist"]
            else:
                raise ValueError(f"""return_ must be either 'dict', 'feature' or 'dist'.
                                     Value given was {return_}""")
        except KeyError:
            raise KeyError(f"{target_type_} hasn't been mapped yet.")


Generating Elevation Map...


DriverError: ../Geospatial/GEOSPATIAL/national-map-line/national-map-line-geojson.geojson: No such file or directory

In [4]:
class Malls(Features):
    
    def __init__(self, complete=False):
        print("Generating Malls...")
        raw_malls_df = sheets.getGeo("Malls Raw")
        if complete:
            malls_dict = raw_malls_df.set_index("Name").to_dict("index")
        else:
            malls_dict = raw_malls_df[
                pd.notna(raw_malls_df.Floors)
                & pd.notna(raw_malls_df.Stores)
                & pd.notna(raw_malls_df.Area)].set_index("Name").to_dict("index")   
        super().__init__(
            [Mall(name, details) for name, details in malls_dict.items()],
            type_="malls")
        print("Done")
        
    def __str__(self):
        return ", ".join(list(self.malls.keys())[:20]) + "..."
    
    def __repr__(self):
        return f"<Malls object containing {len(self.malls)} malls>"

class Mall(Feature):
    
    __slots__ = ("stores", "floors", "year", "area")
    
    def __init__(self, name, mallDict):
        super().__init__(
            name, long=mallDict["Longitude"],
            lat=mallDict["Latitude"],
            geometry=mallDict["Shape"], type_="mall")
        self.floors = mallDict["Floors"]
        self.stores = mallDict["Stores"]
        self.year = mallDict["Opening year"]
        self.area = mallDict["Area"]
        
    def __str__(self):
        return f"""<[Mall] {self.name} object: 
                   Coords: ({round(self.lat, 4)}, {round(self.long, 3)}), 
                   Floors: {int(self.floors)}, 
                   Stores: {int(self.stores)}, 
                   Opening Year: {int(self.year)}, 
                   Retail Area: {self.area}sqft>"""
    
    def __repr__(self):
        return f"<Mall: {self.name}>"


In [5]:
class Schools(Features):
    
    def __init__(self, complete = False):
        print("Generating Schools...")
        raw_schools_df = sheets.getGeo("Schools")
        if complete:
            schools_dict = raw_schools_df.to_dict("records")
        else:
            schools_dict = raw_schools_df[raw_schools_df.Latitude != 0].to_dict("records")
            
        list_of_schools = []    
        for school_dict in schools_dict:
            if school_dict["Level"] == "Primary":
                list_of_schools.append(PrimarySchool(school_dict["Name"], school_dict))
            elif school_dict["Level"] == "Secondary":
                list_of_schools.append(SecondarySchool(school_dict["Name"], school_dict))
            elif school_dict["Level"] == "Tertiary":
                list_of_schools.append(TertiarySchool(school_dict["Name"], school_dict))
        super().__init__(list_of_schools, "schools")
        print("Done")
        
    def __str__(self):
        return ", ".join(list(self.schools.keys())[:20]) + "..."
    
    def __repr__(self):
        return f"<Schools object containing {len(self.schools)} schools>"
    
class School(Feature):
    
    __slots__ = ("code", "funding", "level", "year", "gender")
    
    def __init__(self, name, school_dict):
        super().__init__(
            name, long = school_dict["Longitude"],
            lat = school_dict["Latitude"], type_="school")
        self.code = school_dict["School Code"]
        self.funding = school_dict["Funding"]
        self.level = school_dict["Level"]
        self.year = school_dict["Opening Year"]
        self.gender = school_dict["Type"]
        
    def __str__(self):
        return f"""<[{self.level.capitalize()} School] {self.name} object: 
                   Coords: ({round(self.lat, 4)}, {round(self.long, 3)}), 
                   Level: {self.level}, 
                   School Code: {self.code}, 
                   Opening Year: {int(self.year)}, 
                   Gender Type: {self.gender}>"""
    
    def __repr__(self):
        return f"<{self.level.capitalize()} School: {self.name}>"
    
class PrimarySchool(School):
    
    def __init__(self, name, school_dict):
        super().__init__(name, school_dict)
        
class SecondarySchool(School):
    
    def __init__(self, name, school_dict):
        super().__init__(name, school_dict)
        self.exp = school_dict["Exp"]
        self.na = school_dict["NA"]
        self.nt = school_dict["NT"]
        self.ip = school_dict["IP"]
        
class TertiarySchool(School):
    
    def __init__(self, name, school_dict):
        super().__init__(name, school_dict)
        

In [6]:
class Geometry(ABC):
    
    __slots__ = ("geometry", "center")
    
    def __str__(self):
        pass
    
    def __repr__(self):
        pass
    
    def get_distance_to_feature(self):
        pass
    
class Point(Geometry, shapely.geometry.Point):
    
    def __init__(self, point):
        self.geometry = point
        
    def __str__(self):
        return f"[Point] {str(self.geometry)}"
        
    def __repr__(self):
        return str(self.geometry)
        
    def get_distance_to_feature(self, feature):
        try:
            return DistanceCalculator.get_distance_between(self.geometry, feature.point)
        except AttributeError:
            return float("inf")
        
class Line(Geometry, shapely.geometry.LineString):
    
    def __init__(self, line):
        self.geometry = line
        
    def __str__(self):
        return f"[Line] {str(self.geometry)}"
        
    def __repr__(self):
        return str(self.geometry)
        
    def get_distance_to_feature(self, feature):
        try:
            point = shapely.ops.nearest_points(self.geometry, feature.point)[0]
            return DistanceCalculator.get_distance_between(point, feature.point)
        except AttributeError:
            return float("inf")
    
class Shape(Geometry, shapely.geometry.Polygon):
    
    def __init__(self, shape):
        self.geometry = shape
        
    def __str__(self):
        return f"[Shape] {str(self.geometry)}"
        
    def __repr__(self):
        return str(self.geometry)
        
    def get_distance_to_feature(self, feature):
        try:
            if feature.point.within(self.geometry):
                return 0
            else:
                point = shapely.ops.nearest_points(self.geometry, feature.point)[0]
                return DistanceCalculator.get_distance_between(point, feature.point)
        except AttributeError:
            return float("inf")
    

In [2]:
class Exits(Features):
    
    PATH_TO_TRAIN_STATION_EXITS = "../Geospatial/2022/TrainStationExit_Aug2021/Train_Station_Exit_Layer.shp"
    
    def __init__(self, exits = []):
        if exits == []:
            print("Generating MRT Exits...")
            exits_df = gpd.read_file(self.PATH_TO_TRAIN_STATION_EXITS)
            exits_df["geometry"] = exits_df.geometry.to_crs(epsg=4326)
            exits_dict = exits_df.to_dict("index")
            super().__init__(
                [Exit(name, details) for name, details in exits_dict.items()],
                type_="exits")
            print("Done")
        else:
            for exit in exits:
                exit.name = exit.exitCode.split()[1]
            super().__init__(
                [Exit(exit.exitCode.split()[1], exit) for exit in exits],
                type_="exits")
        
    def __str__(self):
        return ", ".join(list(self.exits.keys())[:20]) + "..."
    
    def __repr__(self):
        return f"<Exits object containing {len(self.features)} exits>"

class Exit(Feature):
    
    __slots__ = ("exit_code")
    
    def __init__(self, name, exit_dict):
        super().__init__(name, long = exit_dict["geometry"].x, lat = exit_dict["geometry"].y, type_="exit")
        self.exit_code = exit_dict["EXIT_CODE_"]
        
    def __str__(self):
        return f"""Exit: {self.exit_code},
                   Coords: ({self.lat}, {self.long})"""
    
    def __repr__(self):
        return f"<Exit: {self.name}>"
    

NameError: name 'Features' is not defined

In [ ]:
class ExistingTrainStations(Features):
    
    PATH_TO_TRAIN_STATIONS = "../Geospatial/2022/TrainStation_Jan2022/MRTLRTStnPtt.shp"
    EXITS = Exits()
    
    def __init__(self):
        print("Generating Existing MRT Stations...")
        stations_df = gpd.read_file(self.PATH_TO_TRAIN_STATIONS)
        stations_df["geometry"] = stations_df.geometry.to_crs(epsg=4326)
        stations_df["STN_NAME"] = stations_df.STN_NAME.str.replace(" .RT STATION", "", regex = True)
        stations_df["STN_NAME"] = stations_df.STN_NAME.str.replace(" STATION", "", regex = True)
        stations_df["STN_NAME"] = stations_df.STN_NAME.apply(
            lambda name: " ".join([word.capitalize() for word in name.split()]))
        stations_df["STN_NAME"] = stations_df.STN_NAME.str.replace("Harbourfront", "HarbourFront")
        stations_df["STN_NAME"] = stations_df.STN_NAME.str.replace("Macpherson", "MacPherson")
        stations_df["STN_NAME"] = stations_df.STN_NAME.str.replace("One-north", "one-north")
        stations_df = stations_df.merge(stations_df.groupby("STN_NAME").geometry.agg(list).reset_index(), on = "STN_NAME")
        stations_df["geometry_y"] = stations_df.geometry_y.apply(
            lambda x: shapely.geometry.Polygon(x) if len(x) > 2
            else shapely.geometry.LineString(x) if len(x) == 2
            else x)
        stations_df = stations_df.groupby("STN_NAME").first()
        stations_dict = stations_df.to_dict("index")
        super().__init__(
            [ExistingTrainStation(name, details) for name, details in stations_dict.items()],
            type_="stations")
        print("Done")
        
        print("Matching exit to station...")
        for name, exit in self.EXITS.exits.items():
            nearest_station = self.get_nearest_to_target(exit)[1]
            nearest_station.add_exit(exit)
        print("Done")
        
    def __str__(self):
        return ", ".join(list(self.stations.keys())[:20]) + "..."
    
    def __repr__(self):
        return f"<ExistingTrainStations object containing {len(self.stations)} stations>"
    

In [9]:
class ExistingTrainStation(Feature):
    
    def __init__(self, name, station_dict):
        super().__init__(
            name, lat=station_dict["geometry_x"].y,
            long=station_dict["geometry_x"].x,
            geometry=station_dict["geometry_y"])
        self.code = station_dict["STN_NO"]
        self.exits = {}
        
    def __str__(self):
        return f"""Station: {self.name},
                   Coords: ({self.lat}, {self.long})"""
    
    def __repr__(self):
        return f"<Train Station: {self.name}>"
    
    def add_exit(self, exit):
        self.exits[exit.exit_code] = exit
        

In [24]:
E = ExistingTrainStations()

Generating Existing MRT Stations...
Done
Matching exit to station...
Done


In [139]:
class TrainNetwork(Features):
    
    def __init__(self, platforms):
        # raw_network_df = sheets.getGeo("MRT")
        super().__init__(platforms, "network")
        self.connect_platforms()
        self.sort_by_number()
        self.complete_network = self.network.copy()
        
    def __str__(self):
        return ", ".join(list(self.network.keys())[:20]) + "..."
    
    def __repr__(self):
        return f"<TrainNetwork object containing {len(self.network)} platforms>"
    
    def connect_platforms(self):
        for name, feature in self.network.items():
            try:
                feature.l_neighbour = self.network[feature.l_neighbour]
                feature.c_l_neighbour = feature.l_neighbour
            except KeyError:
                feature.l_neighbour = None
                
            try:
                feature.h_neighbour = self.network[feature.h_neighbour]
                feature.c_h_neighbour = feature.h_neighbour
            except KeyError:
                feature.h_neighbour = None
                
            if type(feature.h_neighbour) == str:
                None
            if type(feature.l_neighbour) == str:
                None
                
    def sort_by_number(self):
        self.network = dict(sorted(self.network.items(), key=lambda item: item[1].number))
                
    def get_line(self, line):
        if "EW" in line:
            return self.get_with_regex("EW|CG")
        elif "NS" in line:
            return self.get_with_regex("NS")
        elif "NE" in line:
            return self.get_with_regex("NE")
        elif "CC" in line:
            return self.get_with_regex("CC|CE")
        elif "DT" in line:
            return self.get_with_regex("DT")
        elif "TE" in line:
            return self.get_with_regex("TE")
        elif "J" in line:
            return self.get_with_regex("J")
        elif "CR" in line:
            return self.get_with_regex("CR|CG")
    
    def add_platform(self, to_add):
        to_add = self.complete_network[to_add]
        to_add.is_active = True
        first_active_l = to_add.get_first_active_l()
        first_active_h = to_add.get_first_active_h()
        if first_active_l == None and first_active_h == None:
            None
        elif first_active_l == None:
            first_active_h.c_l_neighbour = to_add
            to_add.c_l_neighbour = None
            to_add.c_h_neighbour = first_active_h
        elif first_active_h == None:
            first_active_l.c_h_neighbour = to_add
            to_add.c_l_neighbour = first_active_l
            to_add.c_h_neighbour = None
        else:
            first_active_l.c_h_neighbour = to_add
            first_active_h.c_l_neighbour = to_add
            to_add.c_l_neighbour = first_active_l
            to_add.c_h_neighbour = first_active_h
        self.network[to_add.name] = to_add
        self.sort_by_number()
            
    def remove_platform(self, to_remove):
        to_remove = self.network[to_remove]
        to_remove.is_active = False
        if to_remove.c_l_neighbour == None and to_remove.c_h_neighbour == None:
            None
        elif to_remove.c_l_neighbour == None:
            to_remove.c_h_neighbour.c_l_neighbour = None
        elif to_remove.h_neighbour == None:
            to_remove.c_l_neighbour.c_h_neighbour = None
        else:
            to_remove.c_h_neighbour.c_l_neighbour = to_remove.c_l_neighbour
            to_remove.c_l_neighbour.c_h_neighbour = to_remove.c_h_neighbour
            
        self.network.pop(to_remove.name, None)
        

In [138]:
class TrainPlatform(Feature):
    
    def __init__(self, name, platform_dict):
        super().__init__(name, lat = platform_dict["Lat"], long = platform_dict["Long"])
        self.station = platform_dict["Name"]
        self.year = platform_dict["Opening Year"]
        self.abbr = platform_dict["Abbreviation"]
        self.chinese = platform_dict["Chinese"]
        self.line = re.findall("\D+", self.name)[0]
        self.is_active = True
        
        if self.line in ["STC", "SW", "SE", "PW", "PE", "BP"]:
            self.type_ = "LRT"
        else:
            self.type_ = "MRT"
            
        color_mapper = {
            "STC": "#748477",
            "PTC": "#748477",
            "SW": "#748477",
            "SE": "#748477",
            "PW": "#748477",
            "PE": "#748477",
            "BP": "#748477",
            "EW": "#009645",
            "CG": "#009645",
            "NS": "#D42E12",
            "NE": "#9900AA",
            "CC": "#FA9E0D",
            "CE": "#FA9E0D",
            "DT": "#005EC4",
            "TE": "#9D5B25",
            "JE": "#0099AA",
            "JS": "#0099AA",
            "CR": "#97C616",
            "JW": "#0099AA",
            "CP": "#97C616"
        }
        self.color = color_mapper[self.line]
            
        s_code_remapper = {
            "SW": ["SW1", "SW8"],
            "SE": ["SE1", "SE5"],
            "PW": ["PW1", "PW7"],
            "PE": ["PE1", "PE7"],
            "SW8": ["SW", "SW7"],
            "SE5": ["SE", "SE4"],
            "PW7": ["PW", "PW6"],
            "PE7": ["PE", "PE6"],
            "NS3": ["NS2", "NS3A"],
            "NS3A": ["NS3", "NS4"],
            "NS4": ["NS3A", "NS5"],
            "TE22": ["TE21", "TE22A"],
            "TE22A": ["TE22", "TE23"],
            "TE23": ["TE22A", "TE24"],
            "CC34": ["CC33", "CC4"]}
        
        number = self.name.replace(self.line, "")
        if number == "":
            self.number = 0
        elif "A" in number:
            self.number = int(number[:-1]) + 0.5
        else:
            self.number = int(number)
        
        if self.name in s_code_remapper:
            self.l_neighbour, self.h_neighbour = s_code_remapper[self.name]
        else:
            if self.number == 1:
                self.l_neighbour = self.line
                self.h_neighbour = self.line + "2"
            else:
                self.l_neighbour = self.line + str(self.number - 1)
                self.h_neighbour = self.line + str(self.number + 1)
                
        self.c_l_neighbour = self.l_neighbour
        self.c_h_neighbour = self.h_neighbour
        
    def __str__(self):
        return f"""Platform: {self.name},
                   Coords: ({self.lat}, {self.long})"""
    
    def __repr__(self):
        return f"<Platform: {self.name} {self.station}>"
    
    @property
    def neighbours(self):
        neighbours = [self.l_neighbour, self.h_neighbour]
        return list(filter(lambda x: x is not None, neighbours))
                
    def get_first_active_h(self):
        c_platform = self.h_neighbour
        while True:
            if c_platform == None:
                return None
            elif c_platform.is_active:
                return c_platform
            else:
                c_platform = c_platform.h_neighbour
                
    def get_first_active_l(self):
        c_platform = self.l_neighbour
        while True:
            if c_platform == None:
                return None
            elif c_platform.is_active:
                return c_platform
            else:
                c_platform = c_platform.l_neighbour
                

In [840]:
class TrainStation(Feature):
    
    E_TRAIN_STATIONS = ExistingTrainStations()
    
    def __init__(self, name, platforms):
        self.neighbours = {}
        self.lat = []
        self.long = []
        self.lines = {}
        self.abbr = ""
        self.chinese = ""
        
        self.platforms = {}
        for platform in platforms:
            self.platforms[platform.name] = platform
            self.lat.append(platform.lat)
            self.long.append(platform.long)
            self.abbr = platform.abbr
            self.chinese = platform.chinese
            self.lines[platform.line] = platform.color
            if platform.line not in self.neighbours:
                self.neighbours[platform.line] = [platform]
            else:
                if platform.l_neighbour != None:   
                    self.neighbours[platform.line].append(platform.l_neighbour)
                if platform.h_neighbour != None:
                    self.neighbours[platform.line].append(platform.h_neighbour)
        
        e_train_dict = self.E_TRAIN_STATIONS[self.name]
        
        self.lat = np.mean(self.lat)
        self.long = np.mean(self.long)
        self.geometry = shapely.geometry.Point(self.long, self.lat)
        
        if e_train_dict != {}:
            self.geometry = e_train_dict.geometry.geometry
            self.exits = e_train_dict.exits
            
        super().__init__(
            self.name, lat=self.lat,
            long=self.long, geometry=self.geometry)
        
    def add_platform(self, platform):
        self.platforms[platform.name] = platform
        
    def removePlatform(self, platform):
        self.platforms.pop(platform.name, None)
        
    def getNeighbours(self):
        return [platform.currentHigher]
    

Generating Existing MRT Stations...
Done
Generating MRT Exits...
Done
Matching exit to station...
Done


In [432]:
def get_land_use():
    land_use = gpd.read_file("../Geospatial/2022/mp08-land-use/mp08-land-use-shp/G_MP08_LAND_USE_PL.shp")
    land_use = land_use[["LU_DESC", "geometry"]]
    land_use.geometry.crs = "epsg:3414"
    land_use["geometry"] = land_use.geometry.to_crs("EPSG:4326")
    land_use_grouped = land_use.groupby("LU_DESC").geometry.agg(lambda x: shapely.geometry.MultiPolygon(x.tolist()))
    return land_use_grouped

In [371]:
from shapely import geometry

class Pt(geometry.Point):
    def __init__(self, x, y):
        super().__init__(x, y)
        
    def __str__(self) -> str:
        return f"<Pt: ({self.x}, {self.y})>"
        
    def __repr__(self) -> str:
        return f"({self.x}, {self.y})"
        
    def get_distance(self, point) -> float:
        if isinstance(point, GeoPt):
            raise TypeError("Cannot be GeoPt!")
        return ((self.y-point.y)**2+(self.x-point.x)**2)**0.5
    
    def get_closest_point(self, *points) -> tuple:
        nearest_point = None
        nearest_dist = float("inf")
        for point in points:
            dist = self.get_distance(point)
            if nearest_dist > dist:
                nearest_dist = dist
                nearest_point = point
        return (nearest_point, nearest_dist)
    
class GeoPt(Pt):
    def __init__(self, lat: float, lon: float) -> None:
        super().__init__(lon, lat)
        
    @property
    def lat(self):
        return self.y
    
    @property
    def lon(self):
        return self.x
    
    def __str__(self) -> str:
        return f"<GeoPt: ({self.lat}, {self.lon})>"
        
    def __repr__(self) -> str:
        return f"({self.lat}, {self.lon})"
    
    def get_distance(self, point: Pt) -> float:
        if isinstance(point, GeoPt):
            return DistanceCalculator.get_distance(self, point)
        raise TypeError("Must be a GeoPt!")
        
    def get_distance_basic(self, point) -> float:
        return ((self.y-point.y)**2+(self.x-point.x)**2)**0.5
    
    def get_closest_point(self, *points) -> tuple:
        nearest_point = None
        nearest_dist = float("inf")
        for point in points:
            dist = self.get_distance_basic(point)
            if nearest_dist > dist:
                nearest_dist = dist
                nearest_point = point
        return (nearest_point, self.get_distance(nearest_point))

class Line(geometry.LineString):
    def __init__(self, points: list) -> None:
        super().__init__(points)
        self.points = KDTree()
        self.points.add_node(*points)
    
    def get_nearest(self, point: Pt) -> float:
        return self.points.nearest(point)
    
class Shape(geometry.Polygon):
    def __init__(self, points: list):
        super().__init__(points)
        self.points = KDTree()
        self.points.add_node(*points)
        
    def get_nearest(self, point: Pt) -> float:
        return self.points.nearest(point)
    

In [1]:
from geo.locations.school import Schools
from geo.locations.mall import Malls
from geo.locations.planning import PlanningAreas

In [2]:
S = Schools.get()
M = Malls.get()
P = PlanningAreas.get()

In [3]:
M.map_nearest_to(P)
S.map_nearest_to(P)
M.map_nearest_to(S)
S.map_nearest_to(M)

[('Admiralty Primary School', (<Mall: Causeway Point>, 1.742)),
 ('Admiralty Secondary School', (<Mall: Sun Plaza>, 1.918)),
 ('Ahmad Ibrahim Primary School', (<Mall: Northpoint City>, 0.653)),
 ('Ahmad Ibrahim Secondary School', (<Mall: Canberra Plaza>, 0.81)),
 ('Ai Tong School', (<Mall: Thomson Plaza>, 0.695)),
 ('Alexandra Primary School', (<Mall: Tiong Bahru Plaza>, 0.62)),
 ('Anchor Green Primary School', (<Mall: Buangkok Square>, 0.916)),
 ('Anderson Primary School', (<Mall: Jubilee Square>, 1.558)),
 ('Anderson Secondary School', (<Mall: Jubilee Square>, 0.529)),
 ('Ang Mo Kio Primary School', (<Mall: Jubilee Square>, 0.924)),
 ('Ang Mo Kio Secondary School', (<Mall: Nex>, 0.749)),
 ('Anglican High School', (<Mall: Bedok Point>, 1.262)),
 ('Anglo-Chinese School', (<Mall: United Square>, 0.907)),
 ('Anglo-Chinese School (Barker Road)',
  (<Mall: Velocity @ Novena Square>, 0.904)),
 ('Anglo-Chinese School (Independent)', (<Mall: The Star Vista>, 0.957)),
 ('Anglo-Chinese School (

In [9]:
M.filter(lambda x: x.nearest_location("planning_area").region == "CENTRAL REGION").sort(lambda x: x.nearest_location("school").name)

{'Thomson Plaza': 'Ai Tong School',
 'The Star Vista': 'Anglo-Chinese Junior College',
 'Far East Plaza': 'Anglo-Chinese School (Junior)',
 'ION Orchard': 'Anglo-Chinese School (Junior)',
 'Lucky Plaza': 'Anglo-Chinese School (Junior)',
 'Shaw Centre': 'Anglo-Chinese School (Junior)',
 'The Paragon': 'Anglo-Chinese School (Junior)',
 'Wheelock Place': 'Anglo-Chinese School (Junior)',
 'Alexandra Retail Centre': 'Blangah Rise Primary School',
 'HarbourFront Centre': "CHIJ St. Theresa's Convent",
 '100 AM': 'Cantonment Primary School',
 'Icon Village': 'Cantonment Primary School',
 'Marina Bay Link Mall': 'Cantonment Primary School',
 'Marina One': 'Cantonment Primary School',
 'Tanjong Pagar Centre': 'Cantonment Primary School',
 'Kallang Wave Mall': 'Dunman High School',
 'Leisure Park Kallang': 'Dunman High School',
 'Viva Vista': 'Fairfield Methodist School',
 'City Square Mall': 'Farrer Park Primary School',
 'Mustafa Centre': 'Farrer Park Primary School',
 'Paya Lebar Square': 'Gey

In [11]:
from gsheets import Sheets
s = Sheets.from_files('client_secrets.json','~/storage.json')["1M9Ujc54yZZPlxOX3yxWuqcuJOxzIrDYz4TAFx8ifB8c"]

In [13]:
type(s)

gsheets.models.SpreadSheet